# Filtro de Kalman.

El siguiente notebook de Jupyter implementa un sencillo filtro de Kalman, captura la entrada del ratón y la contamina con valores entre -30 y 30 para cada coordenada. Posteriormente estos datos serán procesados por el filtro y se mostrarán en pantalla.

![Salida del algoritmo](assets/output.png "Salida del algoritmo graficado con Matplotlib")

### Importación de bibliotecas. 

In [ ]:
import numpy as np
from numpy.linalg import inv
from tkinter import *
import matplotlib.pyplot as plt


### Definición de Clases.

### KalmanFilter
Implementación de un filtro de Kalman, con sus respectivas etapas de predicción y corrección. Se utiliza el operador _@_ de numpy para la multiplicación de matrices. 

#### init()

Constructor de clase.

Argumentos: 

+ X: Estimación de estado
+ P: Estimación de covarianza.
+ F: Modelo de transición de estados
+ Q: Covarianza de ruido en el proceso.
+ Z: Mediciones del estado X
+ H: Observaciones del modelo.
+ R: Covarianza de ruido en las observaciones del modelo.

#### predict()
Primer paso del algoritmo, predice el siguiente estado mediante los cálculos de estimación a priori y covarianza del error asociada a la estimación a priori.

Salidas: 
+ X: Estimación del siguiente estado

#### update()
Segunda etapa del algoritmo, realiza la corrección de las predicciones en base a las lecturas recibidas. Esto mediante los cálculos de Ganancia de Kalman, estimación a posteriori, covarianza del error asociada a la estimación a posteriori.

Argumentos:
+ Z: mediciones del estado X

Salidas:
+ X: Correcciones de estado

#### weights()
Actualiza los valores de Q y R

Argumentos: 
+ Q: Nuevos valores de covarianza de ruido del proceso.
+ R: Nuevos valores de covarianza de ruido de las observaciones. 



In [ ]:
class KalmanFilter:
    """
    Simple Kalman filter
    """

    def __init__(self, X, F, Q, Z, H, R, P):
        """
        Initialise the filter class
        Args:
            X: State estimate
            P: Estimate covariance
            F: State transition model
            Q: Process noise covariance
            Z: Measurement of the state X
            H: Observation model
            R: Observation noise covariance
        """
        self.X = X
        self.P = P
        self.F = F
        self.Q = Q
        self.Z = Z
        self.H = H
        self.R = R

    def predict(self):
        """
        Predict the future state
       
        Returns:
            updated X
        """
        # Project the state ahead
        self.X = self.F @ self.X
        self.P = self.F @ self.P @ self.F.T + self.Q

        return self.X

    def update(self, Z):
        """
        Update the Kalman Filter from a measurement
        Args:
            Z: State measurement
        Returns:
            updated X
        """
        K = self.P @ self.H.T @ inv(self.H @ self.P @ self.H.T + self.R)
        self.X += K @ (Z - self.H @ self.X)
        self.P = self.P - K @ self.H @ self.P

        return self.X

    def weights(self, Q, R):
        """Updates the values of Q and R
         Args: 
            Q: Process noise covariance
            R: Observation noise covariances
         """
        self.Q = Q
        self.R = R


### Points
Esta clase almacena los valores de las coordenadas X y Y de un punto dado.

#### init()
Constructor de clase

Argumentos:
x: coordenada x
y: coordenada y

#### repr()
Forma legible del objeto

Salidas: 
Coordenadas en forma de tupla.

In [ ]:
class Points:
    """A simple class that stores the x and y values of a given point"""
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        """
        string representation.

        returns:
            x and y as a tuple.
        """
        return "(" + str(self.x) + "," + str(self.y) + ")"

    

### Parámetros del modelo.

Se inician los valores el modelo para el filtro de Kalman. Posteriormente se crea un objeto de la clase kalman. Las variables *modelTrust* y *noiseLvl* son controladas por un slider que el usuario puede cambiar para visualizar en tiempo real como afectan al modelo. 

Se define la matriz de estados:

$$X = \begin{bmatrix}
x \\
y \\
dx \\ 
dy
\end{bmatrix}$$

Matriz de transición:

$$
F =
\begin{bmatrix}
x_{k+1} \\ y_{k+1} \\ dx_{k+1} \\ dy_{k+1}
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & \Delta t & 0 \\
0 & 1 & 0 & \Delta t \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x_k \\ y_k \\ dx_k \\ dy_k
\end{bmatrix}
$$

S

In [ ]:
modelTrust = 0.0001
noiseLvl = 3

stateMatrix = np.zeros((4, 1))  # [x, y, delta_x, delta_y]
estimateCovariance = np.eye(stateMatrix.shape[0])
transitionMatrix = np.array(
    [[1, 0, 1, 0],
     [0, 1, 0, 1],
     [0, 0, 1, 0],
     [0, 0, 0, 1]])
processNoiseCov = np.identity(4) * modelTrust
measurementStateMatrix = np.zeros((2, 1))
observationMatrix = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0]])
measurementNoiseCov = np.identity(2) * noiseLvl
kalman = KalmanFilter(X=stateMatrix,
                      P=estimateCovariance,
                      F=transitionMatrix,
                      Q=processNoiseCov,
                      Z=measurementStateMatrix,
                      H=observationMatrix,
                      R=measurementNoiseCov)


### Programa principal

Crea una ventana y un lienzo sobre el cual se dibuja el trazo del ratón, el trazo contaminado y el trazo que ha sido procesado por el filtro de Kalman. 


get_x_and_y():
+ Obtiene y almacena las coordenadas del puntero.
+ Contamina y almacena las coordinadas del puntero.

draw():
+ Traza una línea del punto anterior a la coordenada actual del puntero.
+ Traza una línea del punto anterior contaminado a la coordenada actual contaminada del puntero.
+ Predice los valores del siguiente punto utilizando el método *predict* del objeto kalman.
+ Traza una línea del punto anterior limpio a la coordenada actual limpia del puntero.
+ Realiza la corrección con la medición utilizando el método *update*

release():
+ Al soltar el botón primario del ratón gráfica con matplotlib los trazos. 

clear():
+ Limpia el lienzo y los arreglos de objeto points.

infoText():
+ Texto informativo del lienzo

updateVals()
+ Actualiza los valores de ruido y confianza del modelo utilizando el método *weights* del objeto kalman.

Sliders()
+ Gestiona los controles deslizantes e invoca a la función updateVals. 

In [ ]:
app = Tk()
app.title("Mouse Kalman Filter")


canvas = Canvas(app, width=640, height=480, bg='black')
canvas.pack(anchor='nw', fill='both', expand=1)
realcords = []
noiseCords = []
filterCords = []


def get_x_and_y(event):
    global lasx, lasy
    global laspx, laspy
    global lasnx, lasny

    lasx, lasy = event.x, event.y
    laspx, laspy = event.x + \
        np.random.randint(-30, 30), event.y+np.random.randint(-30, 30)
    lasnx, lasny = event.x + \
        np.random.randint(-30, 30), event.y+np.random.randint(-30, 30)


def draw(event):
    global lasx, lasy
    global laspx, laspy
    global lasnx, lasny
    global predict, measure

    canvas.create_line((lasx, lasy, event.x, event.y), fill='red', width=2)
    realcords.append(Points(event.x, event.y))
    # noise points
    xnoise = event.x+np.random.randint(-30, 30)
    ynoise = event.y+np.random.randint(-30, 30)
    canvas.create_line(lasnx, lasny, xnoise+1, ynoise+1, fill='green')
    noiseCords.append(Points(xnoise, ynoise))
    measure = np.array([[xnoise], [ynoise]])

    # plot prediction
    predict = kalman.predict()
    xpred = predict[0, 0].round()
    ypred = predict[1, 0].round()
    canvas.create_line(laspx, laspy, xpred, ypred, fill='white')
    filterCords.append(Points(xpred, ypred))
    kalman.update(measure)

    laspx, laspy = xpred, ypred
    lasnx, lasny = xnoise, ynoise
    lasx, lasy = event.x, event.y


def release(event):

    real = np.array([list([i.x, i.y])for i in realcords])
    noisy = np.array([list([i.x, i.y])for i in noiseCords])
    clean = np.array([list([i.x, i.y])for i in filterCords])
    plt.plot(real[:, 0], -1*real[:, 1], label='Mouse Input', zorder=1)
    plt.plot(noisy[:, 0], -1*noisy[:, 1], label='Noisy Input', zorder=0)
    plt.plot(clean[:, 0], -1*clean[:, 1], label='Clean Output', zorder=1)
    plt.title("Kalman Filter")
    plt.legend()
    plt.show()


def clear(event):
    canvas.delete('all')
    realcords.clear()
    noiseCords.clear()
    filterCords.clear()
    infoText()


def infoText():
    canvas.create_text(50, 20, text="__ Real", fill="red", font=('Helvetica'))
    canvas.create_text(120, 20, text="_ _ Noisy",
                       fill="green", font=('Helvetica'))
    canvas.create_text(200, 20, text="__ Clean",
                       fill="white", font=('Helvetica'))
    canvas.create_text(480, 460, text="Left click and drag. Right or Scroll click to clean",
                       fill="white", font=('Helvetica'))


def updateVals():
    modelTrust = w.get()
    noiseLvl = w1.get()
    processNoiseCov = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]) * modelTrust
    measurementNoiseCov = np.array([
        [1, 0],
        [0, 1]]) * noiseLvl
    kalman.weights(processNoiseCov, measurementNoiseCov)


def sliders():
    global w, w1
    l1 = Label(app, text="Model Trust")
    l1.pack()
    w = Scale(app, from_=0.00001, to=.1, orient=HORIZONTAL,
              resolution=.0001, length=300, variable=modelTrust)
    w.set(.0001)
    w.pack()

    l2 = Label(app, text="Measurements Trust")
    l2.pack()
    w1 = Scale(app, from_=0, to=4, resolution=.0001,
               orient=HORIZONTAL, length=300, variable=noiseLvl)
    w1.set(3)
    w1.pack()

    b1 = Button(app, text="Update Values",
                command=updateVals)
    b1.pack()


sliders()
infoText()


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)
canvas.bind("<Button-3>", clear)

app.mainloop()
